In [1]:
from gpu_rocket import torchrocket
import numpy as np
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
num_examples = 3
num_channels = 20
timesteps = 100

cuda:0


In [3]:
# either: 
# -use the kernels from sktime (and then convert to 32bit if you want 32bit gpu kernels)
# -generate them yourself
# -or use the one I have edited from sktime to make 32bit 

kernels = torchrocket._generate_kernels_sktime_32bit_version(timesteps, 10_000, num_channels, seed=None)
# Note: it is required to convert each of these np arrays to lists if you want to script the model (script doesnt work with np)
# e.g. channel_indices = channel_indices.tolist() etc

In [4]:
X = np.random.rand(num_examples,num_channels,timesteps).astype(np.float32)
X_torch =torch.from_numpy(X.astype(np.float32)).to(device)

In [5]:
torch_rocket_softPPV_10k = torchrocket.TorchRocket(kernels,ppv_ver='softPPV',softppv_param=10_000.)
torch_rocket_softPPV_10k.to(device)
torch_rocket_softPPV_10k.eval()
torch_rocket_softPPV_5 = torchrocket.TorchRocket(kernels,ppv_ver='softPPV',softppv_param=5.)
torch_rocket_softPPV_5.to(device)
torch_rocket_softPPV_5.eval()
torch_rocket_heaviside_PPV = torchrocket.TorchRocket(kernels,ppv_ver='heaviside_PPV')
torch_rocket_heaviside_PPV.to(device)
torch_rocket_heaviside_PPV.eval()
print(f'')

In [6]:
with torch.no_grad():
    out_softPPV_10k = torch_rocket_softPPV_10k(X_torch)
    out_softPPV_5 = torch_rocket_softPPV_5(X_torch)
    out_heaviside_PPV = torch_rocket_heaviside_PPV(X_torch)

In [7]:
# compare to sktime implementation
from sktime.transformations.panel.rocket import _rocket
# Note: sktime rocket uses 64bit floats (we changed the sktime code to use 32bit for our timings)
(weights,lengths,biases,dilations,paddings,
        num_channel_indices,channel_indices) = kernels
weights = weights.astype(np.float64)
biases = biases.astype(np.float64)

kernels_64 = (weights,lengths,biases,dilations,paddings,
    num_channel_indices,channel_indices)

out_sktime_rocket = _rocket._apply_kernels(X.astype(np.float64), kernels_64)

In [8]:
print(f'out_softPPV shape: {out_softPPV_10k.shape}')
print(f'out_softPPV shape: {out_softPPV_5.shape}')
print(f'out_heaviside_PPV shape: {out_heaviside_PPV.shape}')
print(f'out_sktime_rocket shape: {out_sktime_rocket.shape}')

out_softPPV shape: torch.Size([3, 20000])
out_softPPV shape: torch.Size([3, 20000])
out_heaviside_PPV shape: torch.Size([3, 20000])
out_sktime_rocket shape: (3, 20000)


In [9]:
for i in range(out_softPPV_5.shape[1]):
    print(f'{i}: {out_softPPV_5[0][i]},{out_softPPV_10k[0][i]},{out_heaviside_PPV[0][i]},{out_sktime_rocket[0][i]}')

0: 0.29870784282684326,0.5,0.5,0.5
1: 3.6533780097961426,3.6533780097961426,3.6533780097961426,3.653378220927565
2: 0.34101730585098267,0.550000011920929,0.550000011920929,0.55
3: 3.38918399810791,3.38918399810791,3.38918399810791,3.3891839784390525
4: 0.10830727964639664,0.26923078298568726,0.26923078298568726,0.2692307692307692
5: 1.1919224262237549,1.1919224262237549,1.1919224262237549,1.1919224894266587
6: 0.31163662672042847,0.4158453941345215,0.41999998688697815,0.42
7: 5.102686882019043,5.102686882019043,5.102686882019043,5.102687870742832
8: 0.5785762667655945,0.6699999570846558,0.6699999570846558,0.67
9: 7.199226379394531,7.199226379394531,7.199226379394531,7.199228334006887
10: 0.23001478612422943,0.3499999940395355,0.3499999940395355,0.35
11: 2.6193928718566895,2.6193928718566895,2.6193928718566895,2.619392927515293
12: 0.20583322644233704,0.3499999940395355,0.3499999940395355,0.35
13: 2.070164203643799,2.070164203643799,2.070164203643799,2.070164168057507
14: 0.194803372025